# Frame Segmentation Demo

Демонстрация инференса модели и визуализации результатов с векторными подсказками для сдвигов.


In [1]:
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pathlib import Path
import sys

sys.path.insert(0, str(Path.cwd().parent))

from utils.color_mapping import id_to_color


KeyboardInterrupt: 

## Загрузка данных


In [ ]:
# Пути к данным
data_dir = Path('../data')
screenshot_path = data_dir / 'screenshots' / 'page_1.png'
mask_path = data_dir / 'masks' / 'page_1_instance_mask.png'
result_path = data_dir / 'results' / 'page_1.json'
meta_path = data_dir / 'meta' / 'page_1.json'

# Загрузка скриншота
screenshot = None
screenshot_rgb = None
if screenshot_path.exists():
    screenshot = cv2.imread(str(screenshot_path))
    if screenshot is not None:
        screenshot_rgb = cv2.cvtColor(screenshot, cv2.COLOR_BGR2RGB)
    else:
        print(f"Warning: Could not load screenshot: {screenshot_path}")
else:
    print(f"Warning: Screenshot not found: {screenshot_path}")
    print("Run: python scripts/html_generator.py --n 10")
    print("Then: python scripts/playwright_render.py")

# Загрузка маски
mask = None
mask_rgb = None
if mask_path.exists():
    mask = cv2.imread(str(mask_path))
    if mask is not None:
        mask_rgb = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
    else:
        print(f"Warning: Could not load mask: {mask_path}")
else:
    print(f"Warning: Mask not found: {mask_path}")
    print("Note: Masks are optional. COCO converter works directly from metadata.")
    print("To generate masks from COCO: python scripts/coco_to_mask.py")

# Загрузка результатов инференса
if result_path.exists():
    with open(result_path, 'r') as f:
        results = json.load(f)
else:
    results = None
    print(f"Results file not found: {result_path}")
    print("Run inference first: python detectron/infer_and_postprocess.py --weights outputs/model_final.pth")

# Загрузка метаданных
if meta_path.exists():
    with open(meta_path, 'r') as f:
        metadata = json.load(f)
else:
    metadata = None
    print(f"Warning: Metadata not found: {meta_path}")


error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


## Визуализация скриншота и маски


In [ ]:
if screenshot_rgb is not None and mask_rgb is not None:
    fig, axes = plt.subplots(1, 2, figsize=(20, 10))
    
    axes[0].imshow(screenshot_rgb)
    axes[0].set_title('Screenshot', fontsize=16)
    axes[0].axis('off')
    
    axes[1].imshow(mask_rgb)
    axes[1].set_title('Instance Mask', fontsize=16)
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("Cannot visualize: missing screenshot or mask data.")
    print("Please run the data generation pipeline first.")


## Визуализация результатов инференса с векторными подсказками


In [ ]:
if screenshot_rgb is None:
    print("Cannot visualize overlaps: screenshot not loaded.")
    print("Please run the data generation pipeline first.")
elif results and 'overlaps' in results and len(results['overlaps']) > 0:
    fig, ax = plt.subplots(1, 1, figsize=(20, 12))
    ax.imshow(screenshot_rgb)
    
    # Отображение перекрытий и сдвигов
    for overlap in results['overlaps']:
        top_id = overlap['top_id']
        bottom_id = overlap['bottom_id']
        shift_top = overlap['shift_top']
        shift_bottom = overlap['shift_bottom']
        overlap_pixels = overlap['overlap_pixels']
        
        # Найти центры фреймов из метаданных
        top_frame = None
        bottom_frame = None
        
        if metadata:
            for frame in metadata['frames']:
                if frame['id'] == top_id:
                    top_frame = frame
                if frame['id'] == bottom_id:
                    bottom_frame = frame
        
        if top_frame and bottom_frame:
            # Центр верхнего фрейма
            top_center_x = top_frame['x'] + top_frame['w'] / 2
            top_center_y = top_frame['y'] + top_frame['h'] / 2
            
            # Центр нижнего фрейма
            bottom_center_x = bottom_frame['x'] + bottom_frame['w'] / 2
            bottom_center_y = bottom_frame['y'] + bottom_frame['h'] / 2
            
            # Рисовать стрелки для сдвигов
            if abs(shift_top[0]) > 0 or abs(shift_top[1]) > 0:
                ax.arrow(top_center_x, top_center_y, shift_top[0], shift_top[1],
                       head_width=20, head_length=15, fc='red', ec='red', linewidth=2,
                       label=f'Shift top (ID {top_id})')
            
            if abs(shift_bottom[0]) > 0 or abs(shift_bottom[1]) > 0:
                ax.arrow(bottom_center_x, bottom_center_y, shift_bottom[0], shift_bottom[1],
                       head_width=20, head_length=15, fc='blue', ec='blue', linewidth=2,
                       label=f'Shift bottom (ID {bottom_id})')
            
            # Рисовать прямоугольники фреймов
            rect_top = patches.Rectangle((top_frame['x'], top_frame['y']),
                                       top_frame['w'], top_frame['h'],
                                       linewidth=2, edgecolor='red', facecolor='none')
            ax.add_patch(rect_top)
            
            rect_bottom = patches.Rectangle((bottom_frame['x'], bottom_frame['y']),
                                         bottom_frame['w'], bottom_frame['h'],
                                         linewidth=2, edgecolor='blue', facecolor='none')
            ax.add_patch(rect_bottom)
            
            # Текст с информацией
            ax.text(top_center_x, top_center_y - 30, f'Top ID {top_id}\nOverlap: {overlap_pixels}px',
                   bbox=dict(boxstyle='round', facecolor='red', alpha=0.5), fontsize=10)
            ax.text(bottom_center_x, bottom_center_y + 30, f'Bottom ID {bottom_id}',
                   bbox=dict(boxstyle='round', facecolor='blue', alpha=0.5), fontsize=10)
    
    ax.set_title(f'Overlaps and Shift Vectors (Page {results["page_id"]})', fontsize=16)
    ax.axis('off')
    plt.tight_layout()
    plt.show()
    
    # Вывести JSON результаты
    print("\nResults JSON:")
    print(json.dumps(results, indent=2))
else:
    print("No overlap results found. Run inference first:")
    print("python detectron/infer_and_postprocess.py --weights outputs/model_final.pth")


In [ ]:
if results and 'overlaps' in results:
    overlaps = results['overlaps']
    
    print(f"Total overlaps: {len(overlaps)}")
    print(f"Total detections: {results.get('num_detections', 0)}")
    
    if overlaps:
        overlap_pixels_list = [o['overlap_pixels'] for o in overlaps]
        print(f"\nOverlap statistics:")
        print(f"  Min: {min(overlap_pixels_list)} pixels")
        print(f"  Max: {max(overlap_pixels_list)} pixels")
        print(f"  Mean: {np.mean(overlap_pixels_list):.2f} pixels")
        print(f"  Median: {np.median(overlap_pixels_list):.2f} pixels")
        
        # Гистограмма перекрытий
        plt.figure(figsize=(10, 6))
        plt.hist(overlap_pixels_list, bins=20, edgecolor='black')
        plt.xlabel('Overlap Pixels', fontsize=12)
        plt.ylabel('Frequency', fontsize=12)
        plt.title('Distribution of Overlap Sizes', fontsize=14)
        plt.grid(True, alpha=0.3)
        plt.show()
